In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Importing data
trainData = pd.read_csv('/content/drive/My Drive/quora_dataset/train.csv')
testData = pd.read_csv('/content/drive/My Drive/quora_dataset/test.csv')

We have to use classification algorithms as we have target column in the rangee of 0 and 1 as values. So, to make this simplified we are using classification approach.

In [3]:
trainData.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [4]:
testData.head()

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [5]:
# Checking for null values
trainData.isna().sum()

qid              0
question_text    0
target           0
dtype: int64

In [6]:
testData.isna().sum()

qid              0
question_text    0
dtype: int64

Target column is not available in test data so we have to predict target columnn for test data. We are comparing different classification models and **will use best model** to predict for test data.

In [7]:
# Size of dataset
print(f'Training data has {trainData.shape[0]} rows and {trainData.shape[1]} columns.')

Training data has 1306122 rows and 3 columns.


In [8]:
print(f'Test data has {testData.shape[0]} rows and {testData.shape[1]} columns.')

Test data has 375806 rows and 2 columns.


List of tasks needs to be done before doing any predictions. Here we are doing text pre-processing tasks such as:
*   Removing stop words
*   Converting all words into tokens and making all of them in lower case format
*   Lemmatizing all the words so it can be used as it's root form
*   We are lemmatizing it to reduce complexity and scope of our dataset





In [ ]:
# We are importing spacy library for stop words list
from spacy.lang.en.stop_words import STOP_WORDS

In [10]:
print(f"There are {len(STOP_WORDS)} stop words.")

There are 326 stop words.


*   Stop words aree words in english language which does not convey any meaning in the string. Example are like a, an, the and many more.



In [11]:
import spacy.cli
# We are downloading library which is english core and which is very big library 
# and needs to be accessed in order to process our data.
spacy.cli.download('en_core_web_lg')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# As you installed now you can import it directly 
import en_core_web_lg

In [ ]:
import string
punct = string.punctuation
stopWords = list(STOP_WORDS)
parser = en_core_web_lg.load(disable=['tagger', 'ner'])
def spacy_tokenizer(text):
  sentence = parser(text)
  tokenized = [word.lemma_.lower().strip() for word in sentence]
  tokenized = [word for word in tokenized if word not in stopWords and word not in punct]
  tokenized = ' '.join([i for i in tokenized])
  return tokenized

In [ ]:
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [16]:
# This is very much time consuming function as it converts every row of question_text 
# to respective processing function.
trainData['processed_text'] = trainData['question_text'].progress_apply(spacy_tokenizer)

100%|██████████| 1306122/1306122 [1:21:03<00:00, 268.55it/s]


In [17]:
# This is very much time consuming function as it converts every row of question_text 
# to respective processing function.
testData['processed_text'] = testData['question_text'].progress_apply(spacy_tokenizer)

100%|██████████| 375806/375806 [22:35<00:00, 277.26it/s]


In [ ]:
import pandas as pd
trainData = pd.read_csv('/content/drive/My Drive/trainProcessed.csv')
testData = pd.read_csv('/content/drive/My Drive/testProcessed.csv')

In [18]:
# Processed text column added
trainData.head()

,qid,question_text,target,processed_text
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,quebec nationalist province nation 1960s
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,adopt dog encourage people adopt shop
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,doe velocity affect time velocity affect space...
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke use magdeburg hemisphere
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,convert montra helicon d mountain bike change ...


In [19]:
testData.head()

,qid,question_text,processed_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...,woman rude arrogant little bite wealth power
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...,apply rv college engineer bms college engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...,like nurse practitioner
3,000086e4b7e1c7146103,Who are entrepreneurs?,entrepreneur
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?,education good people nowadays


In [ ]:
# Selecting only required columns
trainData = trainData[['qid', 'question_text', 'processed_text', 'target']]
testData = testData[['qid', 'question_text', 'processed_text']]

In [ ]:
# Tfidf vectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Here maxx features is number of features we want to use in our model
def vectorize(text, maxx_features):
    vectorizer = TfidfVectorizer(max_features=maxx_features)
    # This X wil become sparse matrix, please read more about sparse matrix here:
    # https://machinelearningmastery.com/sparse-matrices-for-machine-learning/
    X = vectorizer.fit_transform(text)
    return X

In [23]:
# We reduced max features to 2 ** 8 because of computing issues.
# If using data from trained data
# text = trainData['processed_text'].values.astype("U")
# If using original data from beginning 
text = trainData['processed_text']
X = vectorize(text, 2 ** 8)
X.shape

(1306122, 256)

In [ ]:
# Need to convert sparse matrix to regular numpy matrix
trainFeatures = pd.DataFrame(X.toarray())

In [ ]:
trainFeatures['target'] = trainData['target']

In [ ]:
# Need to generate classification report with different types of metrics
def classification_report(model_name, data, predicted):
  from sklearn.metrics import precision_score, recall_score
  from sklearn.metrics import accuracy_score
  from sklearn.metrics import f1_score
  print(model_name, '\n')
  print("     Precision: ", '{:,.3f}'.format(float(precision_score(data, predicted, average='macro')) * 100), "%")
  print("        Recall: ", '{:,.3f}'.format(float(recall_score(data, predicted, average='macro')) * 100), "%")
  print("      F1 score: ", '{:,.3f}'.format(float(f1_score(data, predicted, average='macro')) * 100), "%")
  print("Accuracy score: ", '{:,.3f}'.format(float(accuracy_score(data, predicted)) * 100), "%")

In [ ]:
# Importing different models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Splitting data in the form of test set as 33% and training set as 77%
X_train, X_test, Y_train, Y_test = train_test_split(trainFeatures.iloc[:,:-1], trainFeatures.iloc[:, -1:], test_size=0.33, random_state=42)

In [ ]:
classifier = LogisticRegression(max_iter=300)

In [32]:
classifier.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
classifier.score(X_test, Y_test)

0.9416872959786182

In [ ]:
# classifier.fit(X_test, Y_test)
logistiicPredicted = classifier.predict(X_test)

In [40]:
# Metric report on logistic regression
classification_report('Logistic Regression', Y_test, logistiicPredicted)

Logistic Regression 

     Precision:  77.232 %
        Recall:  56.333 %
      F1 score:  59.329 %
Accuracy score:  94.169 %


In [ ]:
# Splitting data in the form of test set as 33% and training set as 77%
X_train, X_test, Y_train, Y_test = train_test_split(trainFeatures.iloc[:,:-1], trainFeatures.iloc[:, -1:], test_size=0.33, random_state=42)

In [ ]:
dTC = DecisionTreeClassifier()

In [44]:
dTC.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [45]:
dTC.score(X_test, Y_test)

0.9340542572171657

In [46]:
# Depth of trees
dTC.get_depth()

623

In [ ]:
dTCPredicted = dTC.predict(X_test)

In [48]:
classification_report('Decision Tree Classifier', Y_test, dTCPredicted)

Decision Tree Classifier 

     Precision:  68.404 %
        Recall:  59.455 %
      F1 score:  62.156 %
Accuracy score:  93.405 %


In [ ]:
SGDC = SGDClassifier(max_iter=1000)

In [50]:
SGDC.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
SGDC.score(X_test, Y_test)

0.9398451583565534

In [ ]:
SGDPredicted = SGDC.predict(X_test)

In [57]:
classification_report('SGD Classifier', Y_test, SGDPredicted)

SGD Classifier 

     Precision:  76.007 %
        Recall:  52.333 %
      F1 score:  52.960 %
Accuracy score:  93.985 %


In [ ]:
RFC = RandomForestClassifier()

In [59]:
RFC.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [60]:
RFC.score(X_test, Y_test)

0.9408010282561639

In [ ]:
RFCPredicted = RFC.predict(X_test)

In [62]:
classification_report('RFC Classifier', Y_test, RFCPredicted)

RFC Classifier 

     Precision:  74.589 %
        Recall:  58.729 %
      F1 score:  62.235 %
Accuracy score:  94.080 %


In [ ]:
# By looking at every model we can say that F1 score is good metric in every situation; so by keeping that in 
# reference we are using SGD classifier.

In [ ]:
# Now we will predict our values with SGD classifier as we do not have test data with target column.

In [68]:
# Converting test data and vectorize them
# We reduced max features to 2 ** 8 because of computing issues.
# If using data from trained data
# text = trainData['processed_text'].values.astype("U")
# If using original data from beginning 
textTrain = testData['processed_text']
Y = vectorize(textTrain, 2 ** 8)
Y.shape

(375806, 256)

In [ ]:
# Need to convert sparse matrix to regular numpy matrix
testFeatures = pd.DataFrame(Y.toarray())

In [ ]:
SGDTestPredicted = SGDC.predict(testFeatures)

In [ ]:
temp =  list(SGDTestPredicted)

In [ ]:
tempData = pd.DataFrame(temp)

In [ ]:
tempData.columns = ['target']

In [ ]:
# So this data is final assigned data with accuracy of more than 94%.
testData['target'] = tempData['target']